## Esercitazione 6  del    27 Marzo 2023 


## Esercizio 1 
Si consideri l'equazione quadratica $x^2 + 2px - q = 0 $ con $p=10^5$ e $q=10^{-i}$, i=1,..,10.
Si stabilisca se il problema relativo al calcolo della soluzione $x= -p+\sqrt{p^2+q}$ risulta essere ben
condizionato per tutti i valori di q assegnati;

In [58]:
# Traduce una funzione da simbolica a numerica
from sympy.utilities.lambdify import lambdify
import scipy.linalg as spl
import numpy as np
import sympy as sym
import numpy.linalg as npl
import math

In [59]:
# Definisco simbolicamente i componenti della funzione
q = sym.symbols('q')
p = sym.symbols('p')
f = -p - sym.sqrt(p**2 + q)
df = sym.diff(f, q, 1)
print("f: {}, Derivata di f: {}".format(f, df))
fp = f.subs(p, 10**5)
dfp = df.subs(p, 10**5)
print("f: {}, Derivata di f: {}".format(fp, dfp))

# Traduzione da simboli a numeri: np sta ad indicare la compatibilità con i numpy array
# Ora le funzioni sono lambda e quindi le posso valutare come voglio
fNum = lambdify(q, fp, np)
dfNum = lambdify(q, dfp, np)

# Valutazione del condizionamento
index = 10.0**(-np.arange(1, 11))
print("\nIndici: {}".format(index))

# Per numeri successivi e più piccoli dello spacing non fanno progredire la funzione
# q è rilevante fino ad un certo, quando vai oltre ad un numero più piccolo dello
# spacing, la q viene trascurata perchè è vicina a 0 e l'operazione divene (-p + p)
print("\nSpacing: {}".format(np.spacing((10**5)**2)))

K = np.abs((dfNum(index) * index) / fNum(index))
print("\nK: {}".format(K))

f: -p - sqrt(p**2 + q), Derivata di f: -1/(2*sqrt(p**2 + q))
f: -sqrt(q + 10000000000) - 100000, Derivata di f: -1/(2*sqrt(q + 10000000000))

Indici: [1.e-01 1.e-02 1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08 1.e-09 1.e-10]

Spacing: 1.9073486328125e-06

K: [2.5e-12 2.5e-13 2.5e-14 2.5e-15 2.5e-16 2.5e-17 2.5e-18 2.5e-19 2.5e-20
 2.5e-21]


## Esercizio 2
Scrivere una funzione numpy che calcola la norma infinito e la norma 1 di un vettore ed una matrice  e testarla su vettori e matrici  a propria scelta. Confrontarne i risultati con quelli ottenuti utilizzando la funzione norm di numpy.linalg

(Ricorda la formula della norma infinito e della norma 1 di una matrice
$||A||_\infty= \max_{j=1,n} \sum_{i} |a_{ij}| $
$\quad ||A||_1= \max_{i=1,n} \sum_{j} |a_{ij}| $)

In [60]:
x = np.array([1, 2, 3, -4, -5])
A = np.array([[1, 2, 3, 4, -5], [6, 7, -8, -9, 10], [11, -12, -13, 14, -15]])

print("Senza la funzione norm()")
normXInf = np.max(np.abs(x))
normX1 = np.sum(np.abs(x))
normAInf = np.max(np.sum(np.abs(A), axis=0))
normA1 = np.max(np.sum(np.abs(A), axis=1))
print(normXInf)
print(normX1)
print(normAInf)
print(normA1)

print("\nCon la funzione norm()")
normXInf = npl.norm(x, np.inf)
normX1 = npl.norm(x, 1)
normAInf = npl.norm(A, np.inf)
normA1 = npl.norm(A, 1)
print(int(normXInf))
print(int(normX1))
print(int(normAInf))
print(int(normA1))

Senza la funzione norm()
5
15
30
65

Con la funzione norm()
5
15
65
30


## Esercizio 3
Implementare una funzione che calcola la norma 2 di una  matrice facendo uso della funzione eigvals del pacchetto numpy.linalg, (np.linalg.eigvals(A)). Testarla sulla matrice A=np.array([[4,-1,6],[2,3,-3],[1,-2,9/2]])   e confrontarne i risultati con quelli ottenuti utilizzando la funzione norm di numpy.linalg

In [61]:
A = np.array([[4, -1, 6], [2, 3, -3], [1, -2, 9/2]])
normA2E = np.sqrt(np.max(npl.eigvals(A.T @ A)))
normA2N = npl.norm(A, 2)
print(normA2E)
print(normA2N)

9.056251013341882
9.056251013341878


## Esercizio 4 
Implementare il calcolo dell'indice di condizionamento in norma 2 di una matrice A come rapporto tra la radice dell'autovalore massimo di $A^TA$ e la radice dell'autovalore minimo di $A^TA$.

In [62]:
A = np.array([[4, -1, 6], [2, 3, -3], [1, -2, 9/2]])
K = (np.sqrt(np.max(npl.eigvals(A.T @ A)))) / (np.sqrt(np.min(npl.eigvals(A.T @ A))))
print(K)

433162369.7958386


## Esercizio 5
- Costruire la matrice di Vandermonde A, generata dal vettore x=[1.0,2.0,...,6.0] utilizzando la funzione np.vander(x, increasing=True) di numpy. 
- Calcolarne l'indice di condizionamento in norma 1 senza utilizzare la funzione cond di numpy.linalg e poi confrontarne il valore con quello ottenuto utilizzando la funzione.
- Considerare il sistema lineare Ax=b  con matrice dei coefficienti A e termine noto costruito in maniera tale che la soluzione esatta sia il vettore x=[1,1,1,1,1,1] (ogni sua componente è 1)
- perturbare il vettore dei termini noti della quantità
$$
\delta b = 0.025 \, \ast \,
\left [
\begin{array}{c}
1\\
0\\
0\\
0
\end{array}
\right ]
$$
- Risolvere il sistema con termine noto pertubato $b + \delta b$ ((usando il metodo solve del modulo linalg di Scipy).
- Calcolare l'errore relativo sul termine noto e l'errore relativo sulla soluzione. Cosa si può concludere?

N.B. per il calcolo dell'inversa della matrice di A usare la funzione di numpy.linalg.inv(A).


In [63]:
x = np.arange(1.0, 7.0)
V = np.vander(x, increasing=True)
print(V)
K = npl.norm(V, 1) * npl.norm(npl.inv(V), 1)
KNpl = npl.cond(V, 1)
# Molto alto perchè convivono elementi molto grandi e molto piccoli assieme
print(K)
print(KNpl)

x = np.array([1, 1, 1, 1, 1, 1])
b = V * x
bPert = b + (b * 0.025)
xPert = spl.solve(npl.inv(V), bPert)
print(xPert)

ERb = np.abs((bPert - b) / bPert)
ERx = np.abs((xPert - x) / xPert)
print(ERb)
print(ERx)

[[1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00 1.000e+00]
 [1.000e+00 2.000e+00 4.000e+00 8.000e+00 1.600e+01 3.200e+01]
 [1.000e+00 3.000e+00 9.000e+00 2.700e+01 8.100e+01 2.430e+02]
 [1.000e+00 4.000e+00 1.600e+01 6.400e+01 2.560e+02 1.024e+03]
 [1.000e+00 5.000e+00 2.500e+01 1.250e+02 6.250e+02 3.125e+03]
 [1.000e+00 6.000e+00 3.600e+01 2.160e+02 1.296e+03 7.776e+03]]
1281105.0000000778
1281105.0000000778
[[6.15000000e+00 2.15250000e+01 9.32750000e+01 4.52025000e+02
  2.33187500e+03 1.25060250e+04]
 [6.45750000e+01 3.29025000e+02 1.76812500e+03 9.78772500e+03
  5.52239250e+04 3.15762525e+05]
 [3.73100000e+02 2.05512500e+03 1.15814750e+04 6.62242250e+04
  3.82574075e+05 2.22694063e+06]
 [1.39912500e+03 7.93042500e+03 4.55602250e+04 2.64188625e+05
  1.54247023e+06 9.05298143e+06]
 [4.00365000e+03 2.30225250e+04 1.33630275e+05 7.80887025e+05
  4.58659927e+06 2.70470450e+07]
 [9.56427500e+03 5.54740250e+04 3.24044525e+05 1.90287252e+06
  1.12196551e+07 6.63649585e+07]]
[[0.02439024

## Esercizio 6
Assegnato il sistema lineare $Ax = b$, con
$$
A =
\left [
\begin{array}{ccc}
6 & 63 & 662.2\\
63 & 662.2 & 6967.8\\
662.2 & 6967.8 & 73393.5664
\end{array}
\right ], \qquad
b =
\left [
\begin{array}{c}
1.1\\
2.33\\
1.7
\end{array}
\right ]
$$
- trovare il vettore soluzione $x$ (usando il metodo solve del modulo linalg di Scipy);
- perturbare la matrice dei coefficienti della quantità
$$
\delta A =
0.01 \, \ast \,
\left [ \begin{array}{ccc}
1 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0
\end{array}
\right ]
$$
quindi calcolare l'errore relativo sulla soluzione e confrontarlo con la
perturbazione relativa sui dati di ingresso. Cosa si osserva?

In [64]:
A = np.array([[6, 63, 622.2], [63, 622.2, 6967.8], [622.2, 6967.8, 73393.5664]])
b = np.array([1.1, 2.33, 1.7])
x = spl.solve(A, b)
print(x)

APert = 0.01 * A
bPert = 0.01 * b
xPert = spl.solve(APert, bPert)
print(xPert)

ERx = np.abs((xPert - x) / xPert)
print(ERx)

[ 1.16476722  0.06128465 -0.01566946]
[ 1.16476722  0.06128465 -0.01566946]
[1.71570887e-15 5.32152831e-15 8.85658476e-16]


## Esercizio 7
 
Assegnato il sistema lineare $Ax = b$, con $A$ matrice di Hilbert di ordine
$4$ e $b = [1, 1, 1, 1]^T$,
 - trovare il vettore soluzione $x$ (usando il metodo solve del modulo linalg di Scipy);
 -  perturbare il vettore dei termini noti della quantità
$$
\delta b = 0.01 \, \ast \,
\left [
\begin{array}{c}
1\\
-1\\
1\\
-1
\end{array}
\right ]
$$
quindi calcolare la soluzione del sistema $A x_p= b_p$ con termine noto $b_p=b+ \delta b$.
Calcolare l'errore relativo sulla soluzione e confrontarlo con la perturbazione relativa sui dati di ingresso. Cosa si osserva?

Nota: per la costruzione della matrice di Hilbert usare la funzione hilbert(n) del modulo scipy.linalg
(scipy.linalg.hilbert(n))  dove bisogna specificare l'ordine n della matrice.

In [65]:
H = spl.hilbert(4)
b = np.array([1, 1, 1, 1]).T
x = spl.solve(H, b)
print(x)

bPert = b + (b * 0.025)
xPert = spl.solve(npl.inv(H), bPert)
print(xPert)

ERx = np.abs((xPert - x) / xPert)
print(ERx)

[  -4.   60. -180.  140.]
[2.13541667 1.31541667 0.97375    0.7785119 ]
[  2.87317073  44.61292366 185.85237484 178.83026225]


## Esercizio 8
Si consideri lo sviluppo in serie dell'esponenziale $e^x=1+x+\frac{1}{2!}x^2+\frac{1}{3!}x^3+\cdots =\sum_{k=0}^\infty \frac{x^k}{k!} $. Si calcoli un'approssimazione dell'esponenziale troncando ad N=100 termini la serie, per valori di x che variano da -10 a 10 incluso. Si calcoli  per ogni valore di x l'errore relativo del valore approssimato rispetto al valore esatto, ottenuto mediante la funzione exp della libreria numpy. Cosa si osserva? Pensare ad un algoritmo più stabile ricordando che $e^{-x}=\frac{1}{e^{|x|}}$. 

In [66]:
def relativeError(floating, real):
    return np.abs((floating - real) / real)


def classicExp(x, n):
    result = 0

    for k in range(n):
        result += x**k / np.math.factorial(k)

    return result


def nonNegExp(x, n):
    result = 0

    if x < 0:
        return 1 / nonNegExp(np.abs(x), n)

    for k in range(n):
        result += x**k / np.math.factorial(k)

    return result

# Chiamate a funzione
x = np.arange(-10, 11)
n = 100

for i in x:
    # approxVal = classicExp(i, n)
    approxVal = nonNegExp(i, n)
    trueVal = np.exp(i)
    errRel = relativeError(approx_val, true_val)
    print(f"x={i}: approx={approxVal}, true={trueVal}, relative error={errRel}")


x=-10: approx=9.519909540267049e-05, true=4.5399929762484854e-05, relative error=0.5231054499997769
x=-9: approx=0.00021892321870731475, true=0.00012340980408667956, relative error=0.5231054499997769
x=-8: approx=0.00041116370815887153, true=0.00033546262790251185, relative error=0.5231054499997769
x=-7: approx=0.000961707462445354, true=0.0009118819655545162, relative error=0.5231054499997769
x=-6: approx=0.0025579183957800547, true=0.0024787521766663585, relative error=0.5231054499997769
x=-5: approx=0.0067416991479715286, true=0.006737946999085467, relative error=0.5231054499997769
x=-4: approx=0.018315728500463785, true=0.01831563888873418, relative error=0.5231054499997769
x=-3: approx=0.04978706837273326, true=0.049787068367863944, relative error=0.5231054499997769
x=-2: approx=0.13533528323661273, true=0.1353352832366127, relative error=0.5231054499997769
x=-1: approx=0.3678794411714423, true=0.36787944117144233, relative error=0.5231054499997769
x=0: approx=1.0, true=1.0, relat

## Esercizio 9
Calcolare l'approssimazione della derivata prima di f(x) = sin(x) in x = 1
mediante l'algoritmo del rapporto incrementale (f(x + h) - f(x))/h per valori decrescenti
di h, confrontandolo con il valore fornito dalla funzione di libreria per f'(x)
mediante calcolo dell'errore relativo.